<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Hadoop Lab


---

### Learning Objectives

- Running apache Hadoop on an AWS cluster
- Navigate the Hadoop file system (HDFS)

### Lesson Guide
- [Introduction to Elastic Map Reduce (EMR)](#EMR)
- [EMR pricing](#pricing)
- [EMR cluster](#spin-up)
- [Launch cluster](#launch)
- [Configure web connection](#configure)
    - [Enable ssh access](#access)
    - [Install and configure foxy-proxy](#foxy-proxy)
- [Hadoop](#hadoop)
- [YARN](#yarn)
- [Exploring HDFS from the command line](#guided-practice)
    - [Exercise 1](#ex1)
- [Exploring HDFS from the web interface](#guided-practice2)
    - [Exercise 2](#ex2)
- [Hadoop word count](#guided-practice3)
    - [Exercise 3](#ex3)
- [Hadoop streaming word count](#guided-practice4)
    - [Exercise 4](#ex4)
- [Additional resources](#resources)

<a id='EMR'></a>
## Intro to Elastic Map Reduce (EMR) 

In a previous lesson we have discovered two very important AWS services: EC2 and S3. Now we want to spin up a computer cluster on Amazon. 

**What is a cluster?**

**What is a typical topology for a Big Data computing cluster?**

Amazon Elastic MapReduce was introduced in April 2009 to automate _provisioning_ of the Hadoop cluster, running and terminating jobs, and handling data transfer between EC2 (VM) and S3 (Object Storage). It simplifies the management of a Hadoop cluster, making it available to anyone at the click of a button.

EMR offers several pre-installed software packages including:

- Hadoop
- Hive
- Hue
- Spark
and many others.

<a name="pricing"></a>
## EMR Pricing

EMR Pricing is based on the type of instances forming the cluster and it's divided in tiers. The pricing adds to the cost of spinning up the instances in EC2.

Also, very importantly, costs are calculated in hourly increments, so if we plan to use the cluster for two sessions of half an hour, we should have it up for one hour consecutively instead of spinning it up and down twice.

EMR is not included in the AWS free tier that you've used in the previous class, so it's always a good practice to do some price checking before you spin up a cluster.

We can use the [AWS cost calculator](https://calculator.s3.amazonaws.com/index.html) to estimate the cost of a  three-node cluster with medium size instances `(m3.xlarge)`. The image below shows the cost for one hour: it's slightly more than one dollar.

![](./assets/images/emrcost.png)

If we were to keep the cluster alive for a month, that would result in a pretty high price, that's why it's so convenient to spin up and down clusters as they are needed.

EMR also supports spot Instances. It is recommended to only run the Task Instance Group on spot instances to take advantage of the lower cost while maintaining availability.

<a name="spin-up"></a>
## EMR cluster 

To spin up the cluster, let's first log-in to AWS and go to the EMR service page:

![](./assets/images/emr.png)

<a name="launch"></a>
### Launch Cluster

![](./assets/images/clusterstart.png)

**Remember to choose the key pair you have already stored on your computer.**


![](./assets/images/clusterstarting.png)

**Notice also that like for EC2 we can list the clusters using the Cluster List pane:**

![](./assets/images/clusterlist.png)

**The cluster will take several minutes to boot completely. Press the circular refresh button in the top right of the console summary ("Cluster list") to refresh your view and see if the cluster is ready.**

**In the meantime, let's do a couple of review checks:**

---

**Do you remember how to connect to an instance on EC2?**

**Do you remember which commands we used in AWSCLI?**

---
**Once the cluster is ready we will see it in green:**

![](./assets/images/clusterready.png)

<a name="configure"></a>
## Configure Web Connection 

To monitor how the cluster works, we will use some browser interfaces. Before we can do that, we will have to go trough one more step. In fact, the default security settings for EMR are pretty tight and do not allow for external web connections to our cluster. In order to connect with a browser we will have to set up an _ssh tunnel_, i.e. have our browser communicate with the cluster via an encrypted channel. 

Luckily, Amazon provides us with simple instructions:

![](./assets/images/webconnection.png)

![](./assets/images/sshtunnel.png)

<a name="access"></a>
### In order to follow them we first need to complete two steps:

#### 1. Enable SSH access to our master node. This is done in the Security Groups pane of the EC2 services page.

![](./assets/images/securitygroups.png)

<a id='foxy-proxy'></a>
#### 2. Install and configure Foxy-Proxy as explained [here](https://docs.aws.amazon.com/ElasticMapReduce/latest/ManagementGuide/emr-connect-master-node-proxy.html).

Once we have enabled SSH access, we can go ahead and connect:

```bash
ssh -i ~/.ssh/MyFirstKey.pem -ND 8157 hadoop@<YOUR_MASTER_DNS>
```

Note that this command will not end because it's keeping the tunnel alive.

If the tunnel and Foxy-proxy are well configured, we should be able to connect to several web services. 

<a id='hadoop'></a>
## Hadoop

---

Apache Hadoop is an open-source software framework for distributed storage and distributed processing of very large data sets on computer clusters built from commodity hardware. All the modules in Hadoop are designed with the fundamental assumption that hardware failures are common and should be automatically handled by the framework.

The core of Apache Hadoop consists of a storage part, known as **Hadoop Distributed File System (HDFS)**, and a processing part called MapReduce. Hadoop splits files into large blocks and distributes them across nodes in a cluster.

### HDFS

The Hadoop distributed file system (HDFS) is a distributed, scalable, and portable file-system written in Java for the Hadoop framework. It's the file system supporting Hadoop.


<a id='yarn'></a>
## YARN
---

Yarn is a resource-management platform responsible for managing computing resources in clusters and using them for scheduling of users' applications. The fundamental idea of YARN is to split up the functionalities of resource management and job scheduling/monitoring into separate daemons. The idea is to have a global ResourceManager (RM) and per-application ApplicationMaster (AM).

The ResourceManager and the NodeManager form the data-computation framework. The ResourceManager is the ultimate authority that arbitrates resources among all the applications in the system. The NodeManager is the per-machine framework agent who is responsible for containers, monitoring their resource usage (cpu, memory, disk, network) and reporting the same to the ResourceManager/Scheduler.

The Yarn resource manager offers a web interface that is accessible in our browser at this address:

    http://<YOUR_MASTER_DNS>:8088/

Go ahead and type that in your browser and you should see a screen like this:

![](./assets/images/yarn.png)

This will be useful when we run a hadoop job in order to check the status of advancement.

<a name="guided-practice"></a>

## Exploring HDFS from the command line
---

Hadoop offers a command line interface to navigate the HDFS. The full documentation can be found here:

http://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html

You can explore the content of the HDFS with commands similar to those  used in the shell, for example

```bash
hadoop fs -ls
```

<a id='ex1'></a>
### Exercise 1
Explore HDFS and describe the content of each folder it contains. 

<a name="guided-practice2"></a>
## Exploring HDFS from the web interface
---

Hadoop also offers a web interface to navigate and manage HDFS. It can be found at this address:

    http://<YOUR_MASTER_DNS>:50070
    

and it looks like this:

![](./assets/images/hdfsweb.png)


Click [here](http://docs.aws.amazon.com/emr/latest/ManagementGuide/emr-web-interfaces.html) for further browser interfaces like yarn.

<a id='ex2'></a>
### Exercise 2
Find how you can navigate the HDFS from the web interface. Is the content listed similar to what you were finding with the command line?


<a name="guided-practice3"></a>
## Hadoop word count
Let's create a very short file and count the number of words using Hadoop:

```bash
hadoop fs -mkdir wordcount-input
echo "hello dear world hello" | hadoop fs -put - wordcount-input/hello.txt
```
<a name="ex3"></a>
### Exercise 3:
Run the word count with the following command:

```bash
hadoop jar hadoop-mapreduce.jar \
                  wordcount wordcount-input wordcount-output
```                  
   
![](assets/images/hdwcshell.png)   
![](assets/images/hdwcyarn.png)   
   
   
Check the results by typing:

```bash
hadoop fs -cat wordcount-output/part*
```

You should see:

    dear   1
    hello  2
    world  1

## Hadoop streaming word count
---

Hadoop also offers a streaming interface. The streaming interface will process the data as a stream, one piece at a time, and it requires to be told what to do with each piece of data. This is somewhat similar to what we did with the map-reduce from the shell that we used in the previous class.

Let's use the same python scripts to run a hadoop streaming map-reduce. We have pre-copied those scripts to the folder scripts. You have to copy them to the hadoop file system.

First of all let's copy some data to hdfs. The data folder contains a folder called project_gutenberg. First we have to copy this folder to AWS. Then we have to copy it to hadoop:

```hadoop
hadoop fs -copyFromLocal project_gutenberg project_gutenberg
hadoop fs -copyFromLocal scripts scripts
```

Go ahead and check that it's there:

`http://<YOUR_MASTER_DNS>:50070`
        
Great! Now we should pipe all the data contained in that folder through our scripts with hadoop streaming. First let's make sure that the scripts work by using the shell pipes we learned in the last lesson.

```bash
cat project_gutenberg/pg84.txt | python scripts/mapper.py | sort -k 1 | python scripts/reducer.py | sort -rnk 2 > result.txt
```

Great! They still work. Ok, now let's do hadoop streaming MR:

```hadoop
hadoop jar scripts/hadoop-streaming.jar  \
      -file scripts/mapper.py   \
      -mapper scripts/mapper.py \
      -file scripts/reducer.py  \
      -reducer scripts/reducer.py \
      -input project_gutenberg/* \
      -output output_gutenberg
```

Check the status of your MR job here in the yarn interface and the output in the hdfs file browser.

You can copy the files with the results to your local folder:

```hadoop
hadoop fs -copyToLocal output_gutenberg/part* .
```

<a id='ex4'></a>
### Exercise 4

You have learned how to spin up a cluster running Hadoop and how to submit map reduce job flows to it! Congratulations.

Go ahead and perform the map-reduce word count on the project Gutenberg data using the Hadoop Jar used in exercise 3. You should get the list of words with the counts as output. You can also save that list to a file and open it in Pandas to sort the words by the most frequent.

<a id='terminate'></a>
## Terminate the cluster

**Make sure you terminate your cluster now:**

![](./assets/images/terminate.png)

<a id='resources'></a>
## Additional resources
---

- [Hadoop](http://hadoop.apache.org/)
- [Hadoop command line](http://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html)
- [YARN](http://hadoop.apache.org/docs/current/hadoop-yarn/hadoop-yarn-site/YARN.html)
- [Hadoop Streaming tutorial](http://www.michael-noll.com/tutorials/writing-an-hadoop-mapreduce-program-in-python/)
- [Hadoop Streaming doc](https://hadoop.apache.org/docs/r1.2.1/streaming.html)